## **| 앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [5]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**
___


- 직접 투표 분류기 (hard voting) : 각 분류기의 예측을 모아 가장 많이 선택된 클래스 예측
- 간접 투표 분류기 (soft voting) : 모든 분류기가 클래스의 확률을 예측할 수 있으면 (predict_proba() 메서드가 존재), 개별 분류기의 예측을 평균내어 확률이 가장 높은 클래스를 예측


### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
___

- 학습률을 0.1처럼 너무 낮춰 트리가 너무 많으면 훈련세트에 과대적합되며, 학습률을 좀더 키우고 점차적으로 작아지도록 설정해야함


### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [1]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

In [2]:
# train/valid/test dataset
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val)

####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다. 

In [3]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [4]:
# model fitting
rf_clf = RandomForestClassifier(n_estimators=100,random_state=42)
ext_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(max_iter=10000, random_state=42)
mlp_clf = MLPClassifier(random_state=42)

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [6]:
# model fitting
from sklearn.ensemble import VotingClassifier

models = [('rf',rf_clf), ('ext',ext_clf), ('svm',svm_clf), ('mlp',mlp_clf)]

voting_clf = VotingClassifier(estimators=models, voting='hard')
voting_clf.fit(X_train, y_train)

/Users/jaewon/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('ext', ExtraTreesClassifier(random_state=42)),
                             ('svm',
                              LinearSVC(max_iter=10000, random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [9]:
# model test
from sklearn.metrics import accuracy_score
y_pred = voting_clf.predict(X_val)
accuracy_score(y_val, y_pred)

0.968

####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `set_params`를 활용해보세요

In [10]:
# 각 분류 모델 학습
rf_clf.fit(X_train, y_train)
ext_clf.fit(X_train, y_train)
svm_clf.fit(X_train, y_train)
mlp_clf.fit(X_train, y_train)

/Users/jaewon/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


MLPClassifier(random_state=42)

In [11]:
# 각 분류 모델의 성능 확인
for clf in (rf_clf, ext_clf, svm_clf, mlp_clf):
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__, accuracy_score(y_val, y_pred))

RandomForestClassifier 0.9647238095238095
ExtraTreesClassifier 0.9674666666666667
LinearSVC 0.8568380952380953
MLPClassifier 0.9590095238095238


- Q. 어떤 모델의 성능이 가장 낮나요?
- A. LinearSVC

In [14]:
# 가장 성능이 낮은 모델 제거
del models[2]
voting_clf2 = VotingClassifier(estimators=models, voting='hard')

In [17]:
# model fitting
voting_clf2.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('ext', ExtraTreesClassifier(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [18]:
# 모델 제거 후 성능 확인
y_pred = voting_clf2.predict(X_val)
accuracy_score(y_val, y_pred)

0.9702857142857143

### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [26]:
rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [27]:
rf_pred = rf_clf.predict(X_val)
ext_pred = ext_clf.predict(X_val)
svm_pred = svm_clf.predict(X_val)
mlp_pred = mlp_clf.predict(X_val)

pred = np.array([rf_pred, ext_pred, svm_pred, mlp_pred])
pred = np.transpose(pred)

####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [34]:
rf_clf_blender = RandomForestClassifier(n_estimators=100,random_state=42)
rf_clf_blender.fit(pred, y_val)

RandomForestClassifier(random_state=42)

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다. 

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [35]:
# 각 분류 모델의 예측을 만들어 새로운 데이터셋 생성
rf_pred_t = rf_clf.predict(X_test)
ext_pred_t = ext_clf.predict(X_test)
svm_pred_t = svm_clf.predict(X_test)
mlp_pred_t = mlp_clf.predict(X_test)

In [36]:
# 새로운 데이터셋을 이용하여 블렌더로 예측
pred_t = np.array([rf_pred_t, ext_pred_t, svm_pred_t, mlp_pred_t])
pred_t = np.transpose(pred_t)

y_pred = rf_clf_blender.predict(pred_t)

In [37]:
# model test
accuracy_score(y_test, y_pred)

0.9669714285714286